In [0]:
from pyspark.sql.types import StructType, StructField, StringType,IntegerType,DoubleType,DateType


In [0]:
dbutils.fs.ls("/mnt/formula1dljit/raw")

In [0]:
races_schema=StructType([StructField("raceid",IntegerType()),
                               StructField("year",IntegerType()),
                               StructField("round",IntegerType()),
                               StructField("circuitid",IntegerType()),
                               StructField("name",StringType()),
                               StructField("date",DateType()),
                               StructField("time",StringType()),
                               StructField("url",StringType())])

In [0]:
races_df=spark.read.option("header",True).schema(races_schema).csv("/mnt/formula1dljit/raw/races.csv")

In [0]:
races_df.printSchema()

In [0]:
from pyspark.sql.functions import col,lit,concat,to_timestamp,current_timestamp

In [0]:
races_df_selected=races_df.select(col("raceid"),col("year"),col("round"),col("circuitid"),col("name"),col("date"),col("time"))
display(races_df_selected)

In [0]:
races_df_renamed=races_df_selected.withColumnRenamed("raceid","race_id").\
    withColumnRenamed("year","race_year").\
        withColumnRenamed("circuitid","circuit_id")

In [0]:
display(races_df_renamed)

In [0]:
races_df_transformed=races_df_renamed.\
    withColumn("race_timestamp",to_timestamp(concat(col("date"),lit(' '),col("time")),'yyyy-MM-dd HH:mm:ss')).\
    withColumn("ingestion_date",current_timestamp()).\
    drop("date","time")

In [0]:
display(races_df_transformed)

In [0]:
#races_df_transformed.write.mode("overwrite").partitionBy("race_year").parquet("/mnt/formula1dljit/processed/races")

In [0]:
races_df_transformed.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.races")

In [0]:
display(spark.read.parquet("/mnt/formula1dljit/processed/races"))